# import libraries

In [4]:
from glob import glob
import cv2
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from keras.applications.densenet import DenseNet169
from sklearn.model_selection import cross_validate, KFold
from keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, roc_curve, auc
from google.colab import drive

# Get Dataset

Download dataset first time and save to google drive


In [ ]:
drive.mount('/content/drive')
# ! wget -P /content/drive/MyDrive https://github.com/muhammedtalo/COVID-19/archive/refs/heads/master.zip
# ! unzip -q /content/drive/MyDrive/master.zip -d /content/drive/MyDrive/Dataset

run this cell when dataset exist in google drive

In [ ]:
drive.mount('/content/drive')

# Load Data

load images directory

In [ ]:
images_path = {}
images_path["Covid"] = glob("/content/drive/MyDrive/Dataset/COVID-19-master/X-Ray Image DataSet/Covid-19/*.png")
images_path["Covid"] += glob("/content/drive/MyDrive/Dataset/COVID-19-master/X-Ray Image DataSet/Covid-19/*.jpeg")
images_path["Covid"] += glob("/content/drive/MyDrive/Dataset/COVID-19-master/X-Ray Image DataSet/Covid-19/*.jpg")

images_path["NoFindings"] = glob("/content/drive/MyDrive/Dataset/COVID-19-master/X-Ray Image DataSet/No_findings/*.png")
images_path["NoFindings"] += glob("/content/drive/MyDrive/Dataset/COVID-19-master/X-Ray Image DataSet/No_findings/*.jpeg")
images_path["NoFindings"] += glob("/content/drive/MyDrive/Dataset/COVID-19-master/X-Ray Image DataSet/No_findings/*.jpg")

images_path["Pneumonia"] = glob("/content/drive/MyDrive/Dataset/COVID-19-master/X-Ray Image DataSet/Pneumonia/*.png")
images_path["Pneumonia"] += glob("/content/drive/MyDrive/Dataset/COVID-19-master/X-Ray Image DataSet/Pneumonia/*.jpeg")
images_path["Pneumonia"] += glob("/content/drive/MyDrive/Dataset/COVID-19-master/X-Ray Image DataSet/Pneumonia/*.jpg")

print(images_path["Covid"])
print(images_path["NoFindings"])
print(images_path["Pneumonia"])

images_class = {
    "Covid": 0,
    "Pneumonia": 1,
    "NoFindings": 2
}

In [ ]:
X = []
Y = []

for label in images_path:
    for image_path in images_path[label]:
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224, 224))
        X.append(image)
        Y.append(images_class[label])

plt.imshow(X[0])
plt.show()

In [ ]:
print(np.array(X).shape)
print(np.array(Y).shape)
plt.imshow(X[1000])

# Build model

build model and get features

In [ ]:
x = np.array(X)
y = np.array(Y)

# initial pre trained model
pre_trained_models = {}
pre_trained_models["DenseNet169"] = DenseNet169(include_top=False, input_shape=(224, 224, 3), pooling="avg")


# Split train and test data 

slpit and make flatten features

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=80)

X_train = pre_trained_models["DenseNet169"].predict(X_train)
X_test = pre_trained_models["DenseNet169"].predict(X_test)

# Classification

initial XGBoost classifier

In [ ]:
result = {}

XGB_Classifier = XGBClassifier(learning_rate=0.44, n_estimators=100, random_state=0, seed=0, gamma=0)

fit XGBoost classifier

In [ ]:
XGB_Classifier.fit(X_train, Y_train)

Y_pred = XGB_Classifier.predict(X_test)

print result

In [ ]:
result["Accuracy score"] = accuracy_score(Y_test, Y_pred)
result["confusion matrix"] = confusion_matrix(Y_test, Y_pred)
result["f1 score"] = f1_score(Y_test, Y_pred, average="macro")
result["precision score"] = precision_score(Y_test, Y_pred, average="macro")
result["Sensitivity"] = result["confusion matrix"][0,0] / (result["confusion matrix"][0,0] + result["confusion matrix"][0,1])
result["Specifity"] = result["confusion matrix"][1,1] / (result["confusion matrix"][1,0] + result["confusion matrix"][1,1])

print(f"\n============")
print(f"\nAccuracy score : ", result["Accuracy score"])
print(f"\nconfusion matrix : \n", result["confusion matrix"])
print(f"\nf1 score : ", result["f1 score"])
print(f"\nprecision score : ", result["precision score"])
print(f"\nSpecifity : ", result["Specifity"])
print(f"\nSensitivity : ", result["Sensitivity"])

In [ ]:
from sklearn.metrics import roc_curve, auc

from sklearn import preprocessing
y_test = preprocessing.label_binarize(Y_test, classes=[0, 1])
y_pred = preprocessing.label_binarize(Y_pred, classes=[0, 1])
print(y_test.shape)
print(y_test.shape)

n_classes = list(Y_test.shape)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area

fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_pred.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

plt.figure()
lw = 2
plt.plot(fpr[1], tpr[1], color='darkorange',
         lw=lw, label='ROC curve (AUC = %0.2f)' % roc_auc[1])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig("roc.svg", format="svg")
plt.show()

In [ ]:

import sklearn
import pandas as pd

scoring = ['precision_macro', 'f1_macro', "accuracy"]

rs = 229
cv = KFold(n_splits=5, random_state=rs, shuffle=True)
scores = cross_validate(XGB_Classifier, features, y, cv=cv, scoring=scoring)
scr = pd.DataFrame(scores)
print(f"======== KFold random state {rs} ==========")
print(scr)
print("accuracy avg : ", scr["test_accuracy"].mean())

In [ ]:
f = 0
for train_ix, test_ix in cv.split(features):
    # get data
    train_X, test_X = features[train_ix], features[test_ix]
    train_y, test_y = y[train_ix], y[test_ix]
    # fit model
    model = XGB_Classifier
    model.fit(train_X, train_y)
    # evaluate model
    yhat = model.predict(test_X)
    acc = accuracy_score(test_y, yhat)
    # store score
    print(f"\n confusion matrix fold {f}")
    f = f+1
    print(confusion_matrix(test_y, yhat))